# Finetuning

Finetune the model for 5 epochs with the combined txt file of the human prompt-story data for gen0.

In [1]:
!deepspeed run_clm.py \
    --model_name_or_path distilgpt2 \
    --train_file data/hd/combined0/train_combined0.txt \
    --validation_file data/hd/initial_combined/valid_combined.txt \
    --do_train \
    --do_eval \
    --output_dir ./models/distilgpt2-finetuned_gen0_100 \
    --num_train_epochs 5 \
    --save_strategy epoch \
    --learning_rate 5e-5 \
    --per_device_train_batch_size 4 \
    --gradient_accumulation_steps 4 \
    --deepspeed ds_config.json \
    #--resume_from_checkpoint ./models/distilgpt2-finetuned_gen0/checkpoint-28500


[2024-04-19 20:30:40,128] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-04-19 20:30:40,680] [WARNING] [runner.py:202:fetch_hostfile] Unable to find hostfile, will proceed with training with local resources only.
[2024-04-19 20:30:40,691] [INFO] [runner.py:568:main] cmd = /home/vasi/Documents/BA_Thesis_Experiment/.venv/bin/python -u -m deepspeed.launcher.launch --world_info=eyJsb2NhbGhvc3QiOiBbMCwgMV19 --master_addr=127.0.0.1 --master_port=29500 --enable_each_rank_log=None run_clm.py --model_name_or_path distilgpt2 --train_file data/hd/combined0/train_combined0.txt --validation_file data/hd/initial_combined/valid_combined.txt --do_train --do_eval --output_dir ./models/distilgpt2-finetuned_gen0_75 --num_train_epochs 5 --save_strategy epoch --learning_rate 5e-5 --per_device_train_batch_size 4 --gradient_accumulation_steps 4 --deepspeed ds_config.json
[2024-04-19 20:30:42,551] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_

# Inference
### Step 3
Let the model generate 100 stories from a ChatGPT4-generated prompts and write them to an output file.

In [11]:
with open("data/sd/eval_prompts/eval_prompts.txt") as pfile:
    prompts = pfile.readlines()

print(prompts[3].split('.')[1][1:]+".")

Deep in the Amazon rainforest, a team of scientists discovers a hidden tribe with a secret that could change the course of history.


In [15]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model_name = "./models/distilgpt2-finetuned_gen0_100/checkpoint-11105" 
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

for prompt in prompts:
    inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")


    # Generate text
    output_sequences = model.generate(
        input_ids=inputs,
        attention_mask=None,
        max_length=500,  # determines the maximum length of the generated text
        temperature=0.7,  # controls randomness: lower values make text less random
        top_k=50,  # the K most likely next words are considered for each step
        top_p=0.9,  # only the most probable tokens with probabilities that add up to top_p are considered for each step
        repetition_penalty=1.2,  # penalty applied to repeated words
        do_sample=True,  # set to True to return diverse samples
        num_return_sequences=1,  # number of independently computed samples to generate
        pad_token_id=tokenizer.eos_token_id,
    )

    # Decode the output sequences to get the generated text
    generated_text = tokenizer.decode(output_sequences[0], skip_special_tokens=True)

    #print(generated_text)
    import re

    # Define regex pattern for impurities
    #pattern = r"(<newline>|<newline \d+ :>|<newline\*>|\[.*?\]|“|”|``|''|--|__________________________________________________________________|\*)"

    # Remove the prompt (first sentence) by finding the first period followed by a space or end of text
    text_without_prompt = re.split(r'\.\s*[”]*\s*(?=[A-Z])', generated_text, 1)[1] if '.' in generated_text else generated_text


    # Regex to remove specified impurities
    #cleaned_text = re.sub(r"\<[^\>]*\>|\[WP\]|\-\-", "", text_without_prompt)
    #cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()  # Remove extra spaces and strip leading/trailing spaces
    # Apply regex to remove impurities
    #cleaned_text = re.sub(pattern, "", cleaned_text)

    print(text_without_prompt)

    with open("./outputs/gen0/stories0.txt", "a") as f:
        f.write(text_without_prompt.split("\n")[1] + "\n\n")


In a post-apocalyptic world where nature has reclaimed the cities, a group of survivors seeks refuge in an abandoned library.
I am here to give you my first words.   I'm afraid that there are people out there who will not understand what is going on inside them, and they want me to tell their stories through this lens so as not to offend anyone but themselves or others alike. If only we could have known how it would go down like this! But if only someone had been able to make sense of all those things without being too careful about telling us anything at the beginning... then why shouldn't our ancestors be allowed into this place? Why does everything start with something bad happening outside of our own home every day since everyone else can do nothing except stare up from their grave for some reason other than because he knows better - which means even more importantly right now your family still ca n ’ t stop looking back while reading books until one night when suddenly somebody co

# Evaluation
### Step 5
Evaluate each story in the stories output file on all metrics and write values to df with last line being the average values of each column.
Metrics represent the model's average for all stories it generated.

In [4]:
from metrics.LexicalDiversity.lexical_diversity import *
from metrics.SemanticDiversity.sementic_diversity import *
from metrics.SyntacticDiversity.syntactic_diversity import *
from nltk.tokenize import sent_tokenize
import pandas as pd
import spacy

# Define the column names
columns = ["Distinct-2", "Distinct-3", "Self-BLEU", "OV-TTR", "MS-TTR", "S-DIV-AV", "S-DIV-C", "SYN-DIV"]

# Create an empty DataFrame with these columns
df_eval_gen0 = pd.DataFrame(columns=columns)

# Load a spaCy model for dependency parsing
nlp = spacy.load("en_core_web_sm")

with open("./outputs/gen0/stories0.txt", 'r') as f:
    stories = f.readlines()

for story in stories:
    if story != '\n':
        print(story)
        # Tokenize the text into sentences
        sentences = sent_tokenize(story)
        graphs = construct_dependency_graphs(sentences)
    

        # Tokenize the text into sentences
        sentences = sent_tokenize(story)

        graphs = construct_dependency_graphs(sentences)

        # Example usage: Adding a new row of data to the DataFrame
        new_data = {
            "Distinct-2": calculate_distinct_n(story, 2),
            "Distinct-3": calculate_distinct_n(story, 3),
            "Self-BLEU": 1-calculate_self_bleu(sentences),
            "OV-TTR": calculate_ttr(story, truncate_length=300),
            "MS-TTR": calculate_mean_segmental_ttr(story, segment_size=50),
            "S-DIV-AV": calculate_semantic_diversity(sentences, 'average'),
            "S-DIV-C": calculate_semantic_diversity(sentences, 'centroid'),
            "SYN-DIV": calculate_syntactic_diversity(graphs)
        }

        # Convert new_data dictionary to a DataFrame
        new_row_df = pd.DataFrame([new_data])
        print(new_row_df)
        # Concatenate the new row DataFrame to the original DataFrame
        df_eval_gen0 = pd.concat([df_eval_gen0, new_row_df], ignore_index=True)
        

# Calculate the mean for each column and append as a new row
averages = df_eval_gen0.mean().to_dict()
averages = {key: [value] for key, value in averages.items()}  # Convert each mean value into a list
average_df = pd.DataFrame(averages)  # Create a DataFrame for the averages
average_df.index = ['Average']  # Label the index as 'Average'

# Append the average row to the original DataFrame
df = pd.concat([df_eval_gen0, average_df])

# Specify the file path and name
file_path = './outputs/gen0/eval_table_gen0.csv'

# Write the DataFrame to a CSV file
df.to_csv(file_path, index=False)  # Set index=False to not include row indices in the file

print(f"Data has been written to {file_path}")
# Print the last row (average values)
print("Average values for each metric:")
print(df.iloc[-1])



   Distinct-2  Distinct-3  Self-BLEU  OV-TTR    MS-TTR  S-DIV-AV   S-DIV-C  \
0    0.992629         1.0        1.0    0.84  0.935556  0.862743  0.577697   

    SYN-DIV  
0  0.879911  
A man walks into his shop, and sees that he has been on this whole ordeal for years.   `` Hello? '' asks the guy from behind the counter who seems to be looking at him with a strange look on his face.   `` Uh... hi! I'm here today just so you can see me out there again if anyone is interested or anything else please let us know about it as well. Also we need to keep your order safe before any customers are coming back home after all these months when everyone starts having trouble eating their food without properly paying attention. We also have to take some extra precautions against people trying too hard because those poor guys won't get through them fast enough even though every single one of them got eaten alive by another person getting sick each time someone gets hurt around them but then most like

/home/vasi/Documents/BA_Thesis_Experiment/.venv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/vasi/Documents/BA_Thesis_Experiment/.venv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/vasi/Documents/BA_Thesis_Experiment/.venv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram over

   Distinct-2  Distinct-3  Self-BLEU    OV-TTR    MS-TTR  S-DIV-AV   S-DIV-C  \
0         1.0         1.0        1.0  0.906667  0.971111  0.866218  0.538991   

    SYN-DIV  
0  0.863044  
`` I'm sorry, but it wouldn't be worth the time. ''   `` It was too late! You did not deserve this... ''   `` Why? Because you don't know what we have left of you and why they will never forgive me because that is your story now - in fact, if only there were people who could understand my work better than them before us without knowing anything about myself or any of my characters. Please just take these words out from my mouth so that everyone knows everything on their own terms ; tell yourself something good at life while telling others nothing else-I mean really no one can hear all of this shit unless someone tells ya otherwise. We both need to stop fighting each other right away with our lives together here instead of talking nonsense like this again when things go wrong by ourselves though. Let'

/home/vasi/Documents/BA_Thesis_Experiment/.venv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/vasi/Documents/BA_Thesis_Experiment/.venv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/vasi/Documents/BA_Thesis_Experiment/.venv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram over

   Distinct-2  Distinct-3  Self-BLEU  OV-TTR    MS-TTR  S-DIV-AV  S-DIV-C  \
0    0.995074         1.0        1.0    0.88  0.939683  0.810912  0.49338   

    SYN-DIV  
0  0.856299  
`` You are not welcome here, son. ''   The boy's eyes widened as he spoke again. `` I have come back from my travels with you since your return at this hour! It is only now that we shall meet once more in our journey home... or so it seemed like long ago after all these years of strife between us for what seems to be a great victory over the people of this realm? We will see who wins next time around but before then there would be no need for such an event ; instead we were bound by our own destiny - if only they could unite themselves against each other than ours together until one day when their fate was decided upon them both. So don't worry about any of this nonsense though : while many may disagree with me concerning the decision of whether or how to proceed without further ado, let alone fight amongs

...

   Distinct-2  Distinct-3  Self-BLEU  OV-TTR    MS-TTR  S-DIV-AV   S-DIV-C  \
0    0.994962         1.0        1.0    0.82  0.912188   0.85464  0.550183   

    SYN-DIV  
0  0.839346  
A man is born in an alternate universe. He has never been able find it, but he's finally found something that will help him cope with his own situation and figure out what exactly happened next...   -- -  I am one of those people who makes great decisions when they feel like there are no other options left than to live their lives without them being around anymore! This was my first attempt at writing this piece because as you can see from here on : )   The year 1125/1245 marks the end of time since we last met so many years ago ; It felt strange enough though ( not quite sure how long had passed? We were still young anyway ) even after all these times over our childhood together where everything seemed normal again…well if only now would I have stopped by today anyways..I guess maybe someday sometime to

/home/vasi/Documents/BA_Thesis_Experiment/.venv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/vasi/Documents/BA_Thesis_Experiment/.venv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/vasi/Documents/BA_Thesis_Experiment/.venv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram over

   Distinct-2  Distinct-3  Self-BLEU  OV-TTR    MS-TTR  S-DIV-AV   S-DIV-C  \
0    0.916667    0.961814        1.0    0.94  0.886667  0.838491  0.390794   

    SYN-DIV  
0  0.716622  
`` I don't know if you are aware of it, but.... ''   `` Well then why is there no way to determine this? If we just kill every other species on Earth and let them live forever after they die in their own bodies -- or leave behind all the life left at that point where our ancestors lived thousands years ago as a whole, what kind of immortality would you think's going be like with your head buried under the ground for centuries now without us knowing about how long time passes from death to eternal hell until something happens somewhere else entirely unexpected occurs! You can only imagine yourself sitting here still trying desperately to find out who killed everyone before someone died because nothing happened so quickly : ) ''   `` Why am I worried he did not have any choice anyway - maybe his body was a

/home/vasi/Documents/BA_Thesis_Experiment/.venv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/vasi/Documents/BA_Thesis_Experiment/.venv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/vasi/Documents/BA_Thesis_Experiment/.venv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram over

   Distinct-2  Distinct-3  Self-BLEU  OV-TTR    MS-TTR  S-DIV-AV   S-DIV-C  \
0    0.941704         1.0        1.0    0.86  0.875887  0.774818  0.352786   

    SYN-DIV  
0  0.651582  
Data has been written to ./outputs/gen0/eval_table_gen0.csv
Average values for each metric:
Distinct-2    0.986767
Distinct-3    0.994604
Self-BLEU     0.999679
OV-TTR        0.863941
MS-TTR        0.943699
S-DIV-AV      0.820552
S-DIV-C       0.489090
SYN-DIV       0.824620
Name: Average, dtype: float64


# Generate Synthetic Data
### Step 6
Contribute to the synthetic dataset by producing stories from the finetuned model.
We use 33% of the original prompt data as our prompt list.

In [2]:
def count_entries(filepath):
    """Counts the number of double-newline-separated entries in a file."""
    with open(filepath, 'r', encoding='utf-8') as file:
        content = file.read().strip()
    return len(content.split('\n\n'))

total_entries = count_entries("./data/hd/combined0/train_combined0.txt")
print("Total entries in GEN0 Dataset with real data: ", total_entries)

sd_entries_count= total_entries/3
print("33% \of total entries = ", sd_entries_count)

Total entries in GEN0 Dataset with real data:  109040
33% \of total entries =  36346.666666666664


In [3]:
import random
from transformers import pipeline, set_seed, GPT2LMHeadModel, GPT2Tokenizer
import os

prompts = []
prompt_files = ["train", "test"]
for name in prompt_files:
    # Path to the file with prompts
    file_path = './data/hd/prepro/'+name+'.wp_source'

    # Read prompts from the file removing the initials [ XX ]
    with open(file_path, 'r', encoding='utf-8') as file:
        prompts += ([line.strip()[7:] for line in file.readlines() if line.strip()])

#print(prompts[0:10])
# Randomly select 25% of the prompts
#sample_size = len(prompts) // 4  # 25%
selected_prompts = random.sample(prompts, int(sd_entries_count))

print(len(selected_prompts))
print(selected_prompts[0:10])


36346
['A day after losing your job , you find yourself in the one place you swore you would never go , doing the one thing you swore you would never do , to help the one person you never wanted to see again', 'To my son , I bequeath the contents of the file New Folder on my computer ...', "A man dies and awakens in limbo , which takes the form of a vast , shining grocery store . He 's left to pace the aisles with only the stockboy-archangel to guide him .", "You are a dragon trying to take your first princess and fight your first knight . The job would be much easier however if you did n't have asthma .", 'Curiously Ghastly Creatures - FirstChapter - 4085 words', 'An invasive alien species is taking over the planet . They are human . You are not .', 'you discover your child has the ability to make anything exist , as long as she believes in it enough . You think this is amazing ... Until you realise children tend to believe in a lot of things quite easily ... .', "`` Whoever fights mo

In [8]:
import os

def get_file_size(filename):
    """Returns the size of the file in bytes."""
    return os.path.getsize(filename)

gen0_data_filename = './data/hd/combined0/train_combined0.txt'
gen0_data_size = get_file_size(gen0_data_filename)
print(f"The size of '{gen0_data_filename}' is {gen0_data_size} bytes.")
# 25% of gen0_data_size
gen0_sd_target_size = gen0_data_size // 4
print("Target size for SD File for GEN1: ", gen0_sd_target_size)


The size of './data/hd/combined0/train_combined0.txt' is 307558317 bytes.
Target size for SD File for GEN1:  76889579


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
import re

model_name = "./models/distilgpt2-finetuned_gen0_100/checkpoint-11105"
tokenizer = GPT2Tokenizer.from_pretrained(model_name, padding_side='left')
model = GPT2LMHeadModel.from_pretrained(model_name)

# Define the device based on CUDA availability
device = "cuda" if torch.cuda.is_available() else "cpu"


if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

if torch.cuda.is_available():
    model = torch.nn.DataParallel(model)
    model.cuda()
else:
    model.to("cpu")

batch_size = 16

def generate_text_batch(prompts):
    inputs = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True)
    inputs = {key: value.to(device) for key, value in inputs.items()}  # Move all tensors to the right device
    outputs = model.module.generate(
        **inputs, 
        max_length=500, 
        num_return_sequences=1, 
        temperature=0.7,  # More randomness
        repetition_penalty=1.2,  # Increase penalty to reduce repetitions
        top_k=50, 
        top_p=0.9
    )
    return [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

output_synth_data = './data/sd/gen0/gen0_sd.txt'

try:
    with open(output_synth_data, 'a', encoding='utf-8') as file:
        for i in range(0, len(selected_prompts), batch_size):
            batch_prompts = selected_prompts[i:i + batch_size]
            generated_texts = generate_text_batch(batch_prompts)
            
            for prompt, generated_text in zip(batch_prompts, generated_texts):
                prompt_length = len(tokenizer.encode(prompt))
                #print(prompt)
                # Remove the prompt by slicing the tokens to skip the prompt length
                generated_text_tokens = tokenizer.encode(generated_text)[prompt_length:]
                clean_generated_text = tokenizer.decode(generated_text_tokens, skip_special_tokens=True)

                # Remove leading and ending spaces and special characters
                clean_generated_text = re.sub(r'^[\s\WP]+', '', clean_generated_text)
                clean_generated_text = re.sub(r'^[\s\W]+|[\s\W]+$', '', clean_generated_text)


                output_text = f"{prompt}\n{clean_generated_text}\n\n"
                #print(output_text)
                if get_file_size(output_synth_data) < gen0_sd_target_size:
                    file.write(output_text)
                else:
                    print("Target size reached!")
                    break
    print("Finished generating stories.")
except Exception as e:
    print(f"An error occurred: {e}")

In [9]:
# check total size of generated sd file
print("Target size for GEN0 sd corresponding to 1/4 of original HD data file: ", gen0_sd_target_size)
print("Actual size of GEN0's generated SD file: ", get_file_size(output_synth_data))

Target size for GEN0 sd corresponding to 1/4 of original HD data file:  76889579
Actual size of GEN0's generated SD file:  76898328
